In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import for plotting 
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


In [41]:
app_train = pd.read_csv('train_imputed.csv')
app_test = pd.read_csv('application_test.csv')
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
pos_cash_balance = pd.read_csv('POS_CASH_balance.csv')
previous_app = pd.read_csv('previous_application.csv')
installments_payments = pd.read_csv('installments_payments.csv')
credit_card_balance = pd.read_csv('credit_card_balance.csv')

KeyboardInterrupt: 

In [ ]:
bureau.head()
bureau_balance.head()

In [3]:


# Groupby the client id (SK_ID_CURR), count the number of previous loans, and rename the column
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loan_counts'})
previous_loan_counts.head()



,SK_ID_CURR,previous_loan_counts
0,100001,7
1,100002,8
2,100003,4
3,100004,2
4,100005,3


In [4]:
def normalize_categorical(df, group_var, col_name):
    
    """Computes counts and normalized counts for each observation
    of `group_var` for each unique category in every categorical variable
    
    Parameters 
    ----------
    df - DataFrame for which we will calculate count
    
    group_var  = string
        The variable by which to group the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    col_name = string
            Variable added to the front of column names to keep track of columns
            
            """
    # select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))
    
    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]
    
    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])                                              
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (col_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

In [5]:


bureau_counts = normalize_categorical(bureau, group_var = 'SK_ID_CURR', col_name = 'bureau')
bureau_counts.head()



,bureau_CREDIT_ACTIVE_Active_count,bureau_CREDIT_ACTIVE_Active_count_norm,bureau_CREDIT_ACTIVE_Bad debt_count,bureau_CREDIT_ACTIVE_Bad debt_count_norm,bureau_CREDIT_ACTIVE_Closed_count,bureau_CREDIT_ACTIVE_Closed_count_norm,bureau_CREDIT_ACTIVE_Sold_count,bureau_CREDIT_ACTIVE_Sold_count_norm,bureau_CREDIT_CURRENCY_currency 1_count,bureau_CREDIT_CURRENCY_currency 1_count_norm,...,bureau_CREDIT_TYPE_Microloan_count,bureau_CREDIT_TYPE_Microloan_count_norm,bureau_CREDIT_TYPE_Mobile operator loan_count,bureau_CREDIT_TYPE_Mobile operator loan_count_norm,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_count_norm
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3,0.428571,0,0.0,4,0.571429,0,0.0,7,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100002,2,0.250000,0,0.0,6,0.750000,0,0.0,8,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100003,1,0.250000,0,0.0,3,0.750000,0,0.0,4,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100004,0,0.000000,0,0.0,2,1.000000,0,0.0,2,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100005,2,0.666667,0,0.0,1,0.333333,0,0.0,3,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [16]:
abcd=pd.read_csv(r"C:\Users\Rahul\Desktop\Fina Assignment\home-credit-default-risk\imputed\train_imputed_done.csv")

In [17]:
abcd.isnull().sum()

Unnamed: 0                    0
Unnamed: 0.1                  0
SK_ID_CURR                    0
TARGET                        0
NAME_CONTRACT_TYPE            0
                             ..
AMT_REQ_CREDIT_BUREAU_DAY     0
AMT_REQ_CREDIT_BUREAU_WEEK    0
AMT_REQ_CREDIT_BUREAU_MON     0
AMT_REQ_CREDIT_BUREAU_QRT     0
AMT_REQ_CREDIT_BUREAU_YEAR    0
Length: 124, dtype: int64

In [6]:
# Grouping data  so  that we can merge all the files in 1 dataset

data_bureau_agg=bureau.groupby(by='SK_ID_CURR').mean()
data_credit_card_balance_agg=credit_card_balance.groupby(by='SK_ID_CURR').mean()
data_previous_application_agg=previous_app.groupby(by='SK_ID_CURR').mean()
data_installments_payments_agg=installments_payments.groupby(by='SK_ID_CURR').mean()
data_POS_CASH_balance_agg=pos_cash_balance.groupby(by='SK_ID_CURR').mean()

data_bureau_agg.head()

,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
SK_ID_CURR,,,,,,,,,,,,,
100001,5896633.000,-735.000000,0.0,82.428571,-825.500000,NaN,0.0,207623.571429,85240.928571,0.00000,0.0,-93.142857,3545.357143
100002,6153272.125,-874.000000,0.0,-349.000000,-697.500000,1681.029,0.0,108131.945625,49156.200000,7997.14125,0.0,-499.875000,0.000000
100003,5885878.500,-1400.750000,0.0,-544.500000,-1097.333333,0.000,0.0,254350.125000,0.000000,202500.00000,0.0,-816.000000,NaN
100004,6829133.500,-867.000000,0.0,-488.500000,-532.500000,0.000,0.0,94518.900000,0.000000,0.00000,0.0,-532.000000,NaN
100005,6735201.000,-190.666667,0.0,439.333333,-123.000000,0.000,0.0,219042.000000,189469.500000,0.00000,0.0,-54.333333,1420.500000


In [7]:
def merge(df):
    df = df.join(data_bureau_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2') 
    df = df.join(bureau_counts, on = 'SK_ID_CURR', how = 'left')
    df = df.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')
    df = df.join(data_credit_card_balance_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2')    
    df = df.join(data_previous_application_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2')   
    df = df.join(data_installments_payments_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2') 
    
    return df

train = merge(app_train)
test = merge(app_test)
display(train.head())

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT1,AMT_ANNUITY1,...,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,-25.000000,-17.000000,0.000000,1.038818e+06,1.052632,10.000000,-295.000000,-315.421053,11559.247105,11559.247105
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,-1054.333333,-1047.333333,0.666667,2.290070e+06,1.040000,5.080000,-1378.160000,-1385.320000,64754.586000,64754.586000
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,-724.000000,-714.000000,0.000000,1.564014e+06,1.333333,2.000000,-754.000000,-761.666667,7096.155000,7096.155000
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,182477.500000,182481.750000,0.000000,2.217428e+06,1.125000,4.437500,-252.250000,-271.625000,62947.088438,62947.088438
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,72136.200000,72143.800000,0.600000,2.048985e+06,1.166667,7.045455,-1028.606061,-1032.242424,12666.444545,12214.060227


In [8]:
train.to_csv("kaggle_merge.csv")

In [2]:
all_data=pd.read_csv("kaggle_merge.csv")

In [4]:
all_data.to_csv("kaggle_merge.csv")

In [5]:
all_data.isnull().sum()

SK_ID_CURR                   0
TARGET                       0
NAME_CONTRACT_TYPE           0
CODE_GENDER                  0
FLAG_OWN_CAR                 0
                         ...  
NUM_INSTALMENT_NUMBER    15868
DAYS_INSTALMENT          15868
DAYS_ENTRY_PAYMENT       15876
AMT_INSTALMENT           15868
AMT_PAYMENT              15876
Length: 230, dtype: int64

In [60]:
# Now we will convert days employed and days registration and days id publish to a positive no. 
def correct_birth(df):
    
    df['DAYS_BIRTH'] = round((df['DAYS_BIRTH'] * (-1))/365)
    return df

def convert_abs(df):
    df['DAYS_EMPLOYED'] = abs(df['DAYS_EMPLOYED'])
    df['DAYS_REGISTRATION'] = abs(df['DAYS_REGISTRATION'])
    df['DAYS_ID_PUBLISH'] = abs(df['DAYS_ID_PUBLISH'])
    df['DAYS_LAST_PHONE_CHANGE'] = abs(df['DAYS_LAST_PHONE_CHANGE'])
    return df

# Now we will fill misisng values in OWN_CAR_AGE. 
#Most probably there will be missing values if the person does not own a car. So we will fill with 0

def missing(df):
    
    features = ['previous_loan_counts','NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_AVG','NONLIVINGAREA_MEDI','OWN_CAR_AGE']
    
    for f in features:
        df[f] = df[f].fillna(0 )
    return df

def transform_app(df):
    df = correct_birth(df)
    df = convert_abs(df)
    df = missing(df)
    return df


all_data = transform_app(all_data)

In [27]:
# counting no of phones given by the company and delete the irrelevant features
all_data['NO_OF_CLIENT_PHONES'] = all_data['FLAG_MOBIL'] + all_data['FLAG_EMP_PHONE'] + all_data['FLAG_WORK_PHONE']
all_data.head()

,Unnamed: 0,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT1,...,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,NO_OF_CLIENT_PHONES
0,0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,...,-17.000000,0.000000,1.038818e+06,1.052632,10.000000,-295.000000,-315.421053,11559.247105,11559.247105,2
1,1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,...,-1047.333333,0.666667,2.290070e+06,1.040000,5.080000,-1378.160000,-1385.320000,64754.586000,64754.586000,2
2,2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,...,-714.000000,0.000000,1.564014e+06,1.333333,2.000000,-754.000000,-761.666667,7096.155000,7096.155000,3
3,3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,...,182481.750000,0.000000,2.217428e+06,1.125000,4.437500,-252.250000,-271.625000,62947.088438,62947.088438,2
4,4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,...,72143.800000,0.600000,2.048985e+06,1.166667,7.045455,-1028.606061,-1032.242424,12666.444545,12214.060227,2


In [6]:
# add a feature to determine if client's permanent city does not match with contact/work city
all_data['FLAG_CLIENT_OUTSIDE_CITY'] = np.where((all_data['REG_CITY_NOT_WORK_CITY']==1) & (all_data['REG_CITY_NOT_LIVE_CITY']==1),1,0)

In [9]:
# add a feature to determine if client's permanent city does not match with contact/work region
all_data['FLAG_CLIENT_OUTSIDE_REGION'] = np.where((all_data['REG_REGION_NOT_LIVE_REGION']==1) & (all_data['REG_REGION_NOT_WORK_REGION']==1),1,0)
all_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT1,AMT_ANNUITY1,...,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,NO_OF_CLIENT_PHONES,FLAG_CLIENT_OUTSIDE_CITY,FLAG_CLIENT_OUTSIDE_REGION
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,1.038818e+06,1.052632,10.000000,-295.000000,-315.421053,11559.247105,11559.247105,2,0,0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,2.290070e+06,1.040000,5.080000,-1378.160000,-1385.320000,64754.586000,64754.586000,2,0,0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,1.564014e+06,1.333333,2.000000,-754.000000,-761.666667,7096.155000,7096.155000,3,0,0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,2.217428e+06,1.125000,4.437500,-252.250000,-271.625000,62947.088438,62947.088438,2,0,0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,2.048985e+06,1.166667,7.045455,-1028.606061,-1032.242424,12666.444545,12214.060227,2,0,0


In [62]:
# deleting useless features
def delete(df):
   # useless=['FLAG_MOBIL', 'FLAG_EMP_PHONE' ,'FLAG_WORK_PHONE','REG_CITY_NOT_WORK_CITY','REG_CITY_NOT_LIVE_CITY','REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION']
    #for feature in useless:
     return df.drop(['FLAG_MOBIL', 'FLAG_EMP_PHONE' ,'FLAG_WORK_PHONE','REG_CITY_NOT_WORK_CITY','REG_CITY_NOT_LIVE_CITY','REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION'], axis=1)
def transform(df):
   # df = convert_abs(df)
    df = delete(df)
   
    return df

all_data = transform(all_data)
all_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT1,AMT_ANNUITY1,...,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,-25.000000,-17.000000,0.000000,1.038818e+06,1.052632,10.000000,-295.000000,-315.421053,11559.247105,11559.247105
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,-1054.333333,-1047.333333,0.666667,2.290070e+06,1.040000,5.080000,-1378.160000,-1385.320000,64754.586000,64754.586000
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,-724.000000,-714.000000,0.000000,1.564014e+06,1.333333,2.000000,-754.000000,-761.666667,7096.155000,7096.155000
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,182477.500000,182481.750000,0.000000,2.217428e+06,1.125000,4.437500,-252.250000,-271.625000,62947.088438,62947.088438
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,72136.200000,72143.800000,0.600000,2.048985e+06,1.166667,7.045455,-1028.606061,-1032.242424,12666.444545,12214.060227


In [63]:
# delete Ids

def delete_id(df):
    return df.drop(['SK_ID_CURR', 'SK_ID_PREV','SK_ID_BUREAU'], axis = 1)

all_data = delete_id(all_data)

In [64]:
# handling missing values

def miss_numerical(df):
    
    features = ['previous_loan_counts','NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_AVG','NONLIVINGAREA_MEDI','OWN_CAR_AGE']
    numerical_features = all_data.select_dtypes(exclude = ["object"] ).columns
    #print(numerical_features)
    for f in numerical_features:
        #print(f)
        if f not in features:
            df[f] = df[f].fillna(df[f].median())
      
    return df

def miss_categorical(df):
    
    categorical_features = all_data.select_dtypes(include = ["object"]).columns
    
    for f in categorical_features:
        df[f] = df[f].fillna(df[f].mode()[0])
        
    return df

def transform_feature(df):
    df = miss_numerical(df)
    df = miss_categorical(df)
    #df = fill_cabin(df)
    return df

all_data = transform_feature(all_data)


all_data.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT1,AMT_ANNUITY1,AMT_GOODS_PRICE1,...,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,125.000000,-25.000000,-17.000000,0.000000,1.052632,10.000000,-295.000000,-315.421053,11559.247105,11559.247105
1,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,-1004.333333,-1054.333333,-1047.333333,0.666667,1.040000,5.080000,-1378.160000,-1385.320000,64754.586000,64754.586000
2,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,-694.000000,-724.000000,-714.000000,0.000000,1.333333,2.000000,-754.000000,-761.666667,7096.155000,7096.155000
3,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,91584.000000,182477.500000,182481.750000,0.000000,1.125000,4.437500,-252.250000,-271.625000,62947.088438,62947.088438
4,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,-837.200000,72136.200000,72143.800000,0.600000,1.166667,7.045455,-1028.606061,-1032.242424,12666.444545,12214.060227


In [67]:
#Standardize code

from sklearn import preprocessing   

def encoder(df):
    numerical=[]
    scaler = preprocessing.StandardScaler()
    for col in all_data:
        if ((len(list(all_data[col].unique())) > 50) & (all_data[col].dtype != 'object')):
            numerical.append(col)
            

    features_transform = pd.DataFrame(data= df)
    features_transform[numerical] = scaler.fit_transform(df[numerical])
    display(features_transform.head(n = 5))
    return df

all_data = encoder(all_data)

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT1,AMT_ANNUITY1,AMT_GOODS_PRICE1,...,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1,Cash loans,M,N,Y,0,0.142129,-0.478095,-0.166143,-0.507236,...,-0.477817,-0.715555,-0.793686,-0.884972,0.021116,0.038796,1.043604,1.030480,-0.286987,-0.280018
1,0,Cash loans,F,N,N,0,0.426792,1.725450,0.592683,1.600873,...,-0.496677,-0.725236,-0.803285,1.273748,-0.000425,-0.408082,-0.813163,-0.807996,2.049805,1.891991
2,0,Revolving loans,M,Y,Y,0,-0.427196,-1.152888,-1.404669,-1.092145,...,-0.491494,-0.722129,-0.800180,-0.884972,0.499811,-0.687835,0.256780,0.263667,-0.483044,-0.462250
3,0,Cash loans,F,N,Y,0,-0.142533,-0.711430,0.177874,-0.653463,...,1.049552,1.000921,0.906647,-0.884972,0.144530,-0.466440,1.116887,1.105737,1.970404,1.818189
4,0,Cash loans,M,N,Y,0,-0.199466,-0.213734,-0.361749,-0.068554,...,-0.493885,-0.036863,-0.121367,1.057876,0.215586,-0.229562,-0.213953,-0.201280,-0.238349,-0.253282


In [36]:
#Standardize code
def encoder(df):
    scaler = preprocessing.StandardScaler()
    numerical = df.select_dtypes(exclude = ["object"]).columns
    features_transform = pd.DataFrame(data= df)
    features_transform[numerical] = scaler.fit_transform(df[numerical])
    display(features_transform.head(n = 5))
    return df

all_data = encoder(all_data)


,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT1,AMT_ANNUITY1,AMT_GOODS_PRICE1,...,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,3.374485,Cash loans,M,N,Y,-0.577538,0.142129,-0.478095,-0.166143,-0.507236,...,-0.477817,-0.715555,-0.793686,-0.884972,0.021116,0.038796,1.043604,1.030480,-0.286987,-0.280018
1,-0.296342,Cash loans,F,N,N,-0.577538,0.426792,1.725450,0.592683,1.600873,...,-0.496677,-0.725236,-0.803285,1.273748,-0.000425,-0.408082,-0.813163,-0.807996,2.049805,1.891991
2,-0.296342,Revolving loans,M,Y,Y,-0.577538,-0.427196,-1.152888,-1.404669,-1.092145,...,-0.491494,-0.722129,-0.800180,-0.884972,0.499811,-0.687835,0.256780,0.263667,-0.483044,-0.462250
3,-0.296342,Cash loans,F,N,Y,-0.577538,-0.142533,-0.711430,0.177874,-0.653463,...,1.049552,1.000921,0.906647,-0.884972,0.144530,-0.466440,1.116887,1.105737,1.970404,1.818189
4,-0.296342,Cash loans,M,N,Y,-0.577538,-0.199466,-0.213734,-0.361749,-0.068554,...,-0.493885,-0.036863,-0.121367,1.057876,0.215586,-0.229562,-0.213953,-0.201280,-0.238349,-0.253282


In [41]:
#Normalization code

from sklearn.preprocessing import MinMaxScaler

def encoder(df):
    scaler = MinMaxScaler()
    numerical = df.select_dtypes(exclude = ["object"]).columns
    features_transform = pd.DataFrame(data= df)
    features_transform[numerical] = scaler.fit_transform(df[numerical])
    display(features_transform.head(n = 5))
    return df

all_data = encoder(all_data)

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT1,AMT_ANNUITY1,AMT_GOODS_PRICE1,...,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1.0,Cash loans,M,N,Y,0.0,0.001512,0.090287,0.090032,0.077441,...,0.007942,0.007764,0.007702,0.000000,0.026991,0.065641,0.899966,0.898168,0.004615,0.004615
1,0.0,Cash loans,F,N,N,0.0,0.002089,0.311736,0.132924,0.271605,...,0.004874,0.004968,0.004903,0.666667,0.026667,0.029757,0.528893,0.549439,0.025854,0.025854
2,0.0,Revolving loans,M,Y,Y,0.0,0.000358,0.022472,0.020025,0.023569,...,0.005717,0.005865,0.005808,0.000000,0.034188,0.007293,0.742720,0.752716,0.002833,0.002833
3,0.0,Cash loans,F,N,Y,0.0,0.000935,0.066837,0.109477,0.063973,...,0.256444,0.503525,0.503494,0.000000,0.028846,0.025071,0.914611,0.912443,0.025133,0.025133
4,0.0,Cash loans,M,N,Y,0.0,0.000819,0.116854,0.078975,0.117845,...,0.005328,0.203787,0.203740,0.600000,0.029915,0.044092,0.648645,0.664523,0.005057,0.004877


In [68]:
# Create a label encoder object
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_count = 0


# Iterate through the columns
for col in all_data:
    if all_data[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(all_data[col].unique())) <= 2:
            # Train on the training data
            le.fit(all_data[col])
            # Transform both training and testing data
            all_data[col] = le.transform(all_data[col])
            #test[col] = le.transform(test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1

print('%d columns were label encoded.' % le_count)

4 columns were label encoded.


In [69]:
all_data = pd.get_dummies(all_data)

display(all_data.shape)

(307511, 340)

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT1,AMT_ANNUITY1,AMT_GOODS_PRICE1,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0,0,0,0,0,0,1,0,1,0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0,1,0,0,0,0,0,0,1,0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,0,0,0,0,0,0,0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0,0,0,0,0,0,0,0,0,0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0,0,0,0,0,0,0,0,0,0


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss
from sklearn.metrics import roc_auc_score

In [70]:
#all_data.to_csv("train_test_split_final.csv")

In [47]:
all_data.shape

(307511, 340)

In [6]:
all_data=pd.read_csv("C:\\Users\\Rahul\\Desktop\\Fina Assignment\\home-credit-default-risk\\pd data\\FINAL_TRAIN_MERGED.csv")

In [7]:
all_data.drop(columns='SK_ID_CURR',inplace=True)

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(all_data.drop(columns='TARGET'), all_data['TARGET'], test_size = 0.2, random_state = 200)
print("X Training shape", X_train.shape)
print("X Testing shape", X_test.shape)
print("Y Training shape", Y_train.shape)
print("Y Testing shape", Y_test.shape)

X Training shape (58234, 395)
X Testing shape (14559, 395)
Y Training shape (58234,)
Y Testing shape (14559,)


In [72]:
Y_train.head()

129821    0
117550    0
66029     0
104850    0
46093     0
Name: TARGET, dtype: int64

In [10]:
logreg = LogisticRegression(random_state=0, class_weight='balanced')
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict_proba(X_test)[:,1]
#Y_pred_proba = logreg.predict_proba(X_test)

print('Train/Test split results:')
#print(logreg.__class__.__name__+" accuracy is %2.3f" % accuracy_score(Y_test, Y_pred))
print("ROC",  roc_auc_score(Y_test, Y_pred))

Train/Test split results:
ROC 0.658213728389167


In [21]:
Y_pred1=Y_pred.apply(lambda x: 0 if x <0.5 else 1)

AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [15]:
Y_test.head()

45638    0
53327    0
9307     0
56592    0
56118    1
Name: TARGET, dtype: int64

In [16]:
output=pd.DataFrame({'actual':Y_test,'pred':Y_pred})

In [18]:
output.to_csv("output.csv")

In [17]:
output.head()

,actual,pred
45638,0,0.508183
53327,0,0.541925
9307,0,0.265463
56592,0,0.447671
56118,1,0.723022


In [12]:
print(Y_test.shape)

Y_test2=Y_test.copy()
Y_test2['pred']=Y_pred
Y_tes2['actual']=
Y_test2.shape

(14559,)


(14560,)

In [14]:
Y_test2.head()

45638    0
53327    0
9307     0
56592    0
56118    1
Name: TARGET, dtype: object

In [50]:
def calculate_woe_iv(dataset, feature, target):
    lst = []
    for i in range(dataset[feature].nunique()):
        val = list(dataset[feature].unique())[i]
        lst.append({
            'Value': val,
            'All': dataset[dataset[feature] == val].count()[feature],
            'Good': dataset[(dataset[feature] == val) & (dataset[target] == 0)].count()[feature],
            'Bad': dataset[(dataset[feature] == val) & (dataset[target] == 1)].count()[feature]
        })
        
    dset = pd.DataFrame(lst)
    dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
    dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
    dset['WoE'] = np.log(dset['Distr_Good'] / dset['Distr_Bad'])
    dset = dset.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WoE']
    iv = dset['IV'].sum()
    
    dset = dset.sort_values(by='WoE')
    
    return dset, iv

In [54]:
for col in all_data.columns:
    if col == 'Exited': continue
    else:
        print('WoE and IV for column: {}'.format(col))
        df, iv = calculate_woe_iv(all_data, col, 'TARGET')
        print(df)
        print('IV score: {:.2f}'.format(iv))
        print('\n')

WoE and IV for column: TARGET
   Value     All    Good    Bad  Distr_Good  Distr_Bad  WoE   IV
0    1.0   24825       0  24825         0.0        1.0  0.0 -0.0
1    0.0  282686  282686      0         1.0        0.0  0.0  0.0
IV score: 0.00


WoE and IV for column: NAME_CONTRACT_TYPE
   Value     All    Good    Bad  Distr_Good  Distr_Bad       WoE        IV
0      0  278232  255011  23221      0.9021   0.935388 -0.036236  0.001206
1      1   29279   27675   1604      0.0979   0.064612  0.415543  0.013833
IV score: 0.02


WoE and IV for column: FLAG_OWN_CAR
   Value     All    Good    Bad  Distr_Good  Distr_Bad       WoE        IV
0      0  202924  185675  17249    0.656824   0.694824 -0.056242  0.002137
1      1  104587   97011   7576    0.343176   0.305176  0.117353  0.004459
IV score: 0.01


WoE and IV for column: FLAG_OWN_REALTY
   Value     All    Good    Bad  Distr_Good  Distr_Bad       WoE        IV
1      0   94199   86357   7842    0.305487   0.315891 -0.033490  0.000348
0      

KeyboardInterrupt: 

In [35]:
# import packages
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv) 

In [36]:
all_data=pd.read_csv("train_imputed.csv")

In [37]:
final_iv, IV = data_vars(all_data,all_data.TARGET)

In [38]:
IV.sort_values('IV',ascending = False)

,VAR_NAME,IV
20,EXT_SOURCE_2,3.185199e-01
21,EXT_SOURCE_3,2.983691e-01
13,DAYS_BIRTH,8.492034e-02
61,ORGANIZATION_TYPE,7.336775e-02
57,NAME_INCOME_TYPE,5.832250e-02
...,...,...
4,AMT_REQ_CREDIT_BUREAU_DAY,2.780785e-08
37,FLAG_DOCUMENT_4,7.821510e-09
23,FLAG_DOCUMENT_10,6.131869e-10
25,FLAG_DOCUMENT_12,5.005563e-11


In [39]:
IV.to_csv("IV values_train_imputed.csv")

In [20]:
train_data=all_data.loc[:,["EXT_SOURCE_2","EXT_SOURCE_2","DAYS_CREDIT","EXT_SOURCE_1","DAYS_BIRTH","bureau_CREDIT_ACTIVE_Closed_count_norm"
                        ,'DAYS_CREDIT_UPDATE','bureau_CREDIT_ACTIVE_Active_count_norm','DAYS_CREDIT_ENDDATE',
                           'NAME_EDUCATION_TYPE_Higher education','DAYS_LAST_PHONE_CHANGE',
                           'AMT_CREDIT_SUM_DEBT','DAYS_DECISION','DAYS_ENTRY_PAYMENT','CNT_DRAWINGS_ATM_CURRENT','TARGET']]

In [24]:
all_data.head()

,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT1,AMT_ANNUITY1,AMT_GOODS_PRICE1,REGION_POPULATION_RELATIVE,...,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden
0,1.0,0,0,1,0.0,0.001512,0.090287,0.090032,0.077441,0.256321,...,1,0,0,0,0,0,0,0,1,0
1,0.0,0,0,0,0.0,0.002089,0.311736,0.132924,0.271605,0.045016,...,1,0,0,1,0,0,0,0,0,0
2,0.0,1,1,1,0.0,0.000358,0.022472,0.020025,0.023569,0.134897,...,1,0,0,0,0,0,0,1,0,0
3,0.0,0,0,1,0.0,0.000935,0.066837,0.109477,0.063973,0.107023,...,1,0,0,0,0,0,0,1,0,0
4,0.0,0,0,1,0.0,0.000819,0.116854,0.078975,0.117845,0.392880,...,1,0,0,0,0,0,0,1,0,0


In [21]:
train_data.isnull().sum()

EXT_SOURCE_2                              0
EXT_SOURCE_2                              0
DAYS_CREDIT                               0
EXT_SOURCE_1                              0
DAYS_BIRTH                                0
bureau_CREDIT_ACTIVE_Closed_count_norm    0
DAYS_CREDIT_UPDATE                        0
bureau_CREDIT_ACTIVE_Active_count_norm    0
DAYS_CREDIT_ENDDATE                       0
NAME_EDUCATION_TYPE_Higher education      0
DAYS_LAST_PHONE_CHANGE                    0
AMT_CREDIT_SUM_DEBT                       0
DAYS_DECISION                             0
DAYS_ENTRY_PAYMENT                        0
CNT_DRAWINGS_ATM_CURRENT                  0
TARGET                                    0
dtype: int64

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(train_data.drop(columns='TARGET'), train_data['TARGET'], test_size = 0.2, random_state = 200)
print("X Training shape", X_train.shape)
print("X Testing shape", X_test.shape)
print("Y Training shape", Y_train.shape)
print("Y Testing shape", Y_test.shape)

NameError: name 'train_data' is not defined

In [21]:
X_test.iloc[:,1]

147004    0
191933    0
149862    0
277630    1
204184    0
         ..
139016    0
250785    0
130788    0
68051     0
16287     0
Name: NAME_CONTRACT_TYPE, Length: 61503, dtype: int64

In [22]:
model = LogisticRegression(random_state=0, class_weight='balanced', C=100)
model.fit(X_train, Y_train)
Y_pred = model.predict_proba(X_test)[:,1]

#Y_pred_proba = logreg.predict_proba(X_test)

print('Train/Test split results:')
#print(logreg.__class__.__name__+" accuracy is %2.3f" % accuracy_score(Y_test, Y_pred))
print("ROC",  roc_auc_score(Y_test, Y_pred))

Train/Test split results:


ValueError: bad input shape (61503, 2)

In [ ]:
model = LogisticRegression(random_state=0, class_weight='balanced', C=100)
model.fit(X_train, Y_train)
Y_pred = model.predict_proba(X_test)

#Y_pred_proba = logreg.predict_proba(X_test)

#print('Train/Test split results:')
#print(logreg.__class__.__name__+" accuracy is %2.3f" % accuracy_score(Y_test, Y_pred))
#print("ROC",  roc_auc_score(Y_test, Y_pred))

In [13]:
model.predict_proba(X_test)[:,1]

array([0.4991217 , 0.49885327, 0.49910463, ..., 0.49921859, 0.49959342,
       0.49990269])

In [33]:
concordance_discordance(X_test,Y_test)

KeyError: "None of [Index([(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)], dtype='object')] are in the [columns]"

In [34]:
#THE FOLLOWING CODE CALCULATES CONCORDANCE AND DISCORDANCE
def concordance_discordance(data,outcome):
    Probability = model.predict_proba(data)
    Probability1 = pd.DataFrame(Probability)
    Probability1.columns = ['Prob_0','Prob_1']
    try:
        TruthTable = pd.merge(data[[outcome]], Probability1[['Prob_1']], how='inner', left_index=True, right_index=True)
    
    zeros = TruthTable[(TruthTable[outcome]==0)].reset_index().drop(['index'], axis = 1)
    ones = TruthTable[(TruthTable[outcome]==1)].reset_index().drop(['index'], axis = 1)
    from bisect import bisect_left, bisect_right
    zeros_list = sorted([zeros.iloc[j,1] for j in zeros.index])
    zeros_length = len(zeros_list)
    disc = 0
    ties = 0
    conc = 0
    for i in ones.index:
        cur_conc = bisect_left(zeros_list, ones.iloc[i,1])
        cur_ties = bisect_right(zeros_list, ones.iloc[i,1]) - cur_conc
        conc += cur_conc
        ties += cur_ties
    pairs_tested = zeros_length * len(ones.index)
    disc = pairs_tested - conc - ties
    print('Pairs = ', pairs_tested)
    print('Conc = ', conc)
    print('Disc = ', disc)
    print('Tied = ', ties)
    concordance = round(conc/pairs_tested,2)
    discordance = round(disc/pairs_tested,2)
    ties_perc = round(ties/pairs_tested,2)
    Somers_D = round((conc - disc)/pairs_tested,2)
    print('Concordance = ', concordance, '%')
    print('Discordance = ', discordance, '%')
    print('Tied = ', ties_perc, '%')
    print('Somers D = ', Somers_D)

SyntaxError: invalid syntax (<ipython-input-34-847aaab4fb77>, line 9)